In [1]:
# Import main packages
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# Create Spark context
sparkConf = SparkConf()
sparkConf.setAppName("wiki")
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
sc = spark.sparkContext

22/12/09 15:16:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# Read dataframe categories.tsv
df_cat = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .option("mode", "DROPMALFORMED") \
    .load("hdfs://localhost:9000/user/bigdata2022/datasets/wiki/categories.tsv")
df_cat.show(5)

+--------------------+--------------------+
|             article|            category|
+--------------------+--------------------+
|%C3%81ed%C3%A1n_m...|subject.History.B...|
|%C3%81ed%C3%A1n_m...|subject.People.Hi...|
|          %C3%85land|   subject.Countries|
|          %C3%85land|subject.Geography...|
|  %C3%89douard_Manet|subject.People.Ar...|
+--------------------+--------------------+
only showing top 5 rows



In [4]:
# Unquote article and category
from urllib.parse import unquote
rdd_cat = df_cat.rdd
rdd_cat  = rdd_cat.map(lambda x: (unquote(x.article),unquote(x.category)))
rdd_cat.collect()

[('Áedán_mac_Gabráin',
  'subject.History.British_History.British_History_1500_and_before_including_Roman_Britain'),
 ('Áedán_mac_Gabráin', 'subject.People.Historical_figures'),
 ('Åland', 'subject.Countries'),
 ('Åland', 'subject.Geography.European_Geography.European_Countries'),
 ('Édouard_Manet', 'subject.People.Artists'),
 ('Éire', 'subject.Countries'),
 ('Éire', 'subject.Geography.European_Geography.European_Countries'),
 ('Óengus_I_of_the_Picts',
  'subject.History.British_History.British_History_1500_and_before_including_Roman_Britain'),
 ('Óengus_I_of_the_Picts', 'subject.People.Historical_figures'),
 ('€2_commemorative_coins', 'subject.Business_Studies.Currency'),
 ('10th_century', 'subject.History.General_history'),
 ('11th_century', 'subject.History.General_history'),
 ('12th_century', 'subject.History.General_history'),
 ('13th_century', 'subject.History.General_history'),
 ('14th_century', 'subject.History.General_history'),
 ('15th_Marine_Expeditionary_Unit',
  'subject.H

In [5]:
# Group the categories per article in an array 
rdd_cat = rdd_cat.map(lambda x: (x[0],[x[1]])) \
            .reduceByKey(lambda a, b: a+b)
rdd_cat.collect()

[('Áedán_mac_Gabráin',
  ['subject.History.British_History.British_History_1500_and_before_including_Roman_Britain',
   'subject.People.Historical_figures']),
 ('Åland',
  ['subject.Countries',
   'subject.Geography.European_Geography.European_Countries']),
 ('Édouard_Manet', ['subject.People.Artists']),
 ('Éire',
  ['subject.Countries',
   'subject.Geography.European_Geography.European_Countries']),
 ('Óengus_I_of_the_Picts',
  ['subject.History.British_History.British_History_1500_and_before_including_Roman_Britain',
   'subject.People.Historical_figures']),
 ('€2_commemorative_coins', ['subject.Business_Studies.Currency']),
 ('10th_century', ['subject.History.General_history']),
 ('11th_century', ['subject.History.General_history']),
 ('12th_century', ['subject.History.General_history']),
 ('13th_century', ['subject.History.General_history']),
 ('14th_century', ['subject.History.General_history']),
 ('15th_Marine_Expeditionary_Unit',
  ['subject.History.Military_History_and_War']),


In [6]:
# Read dataframe links.tsv
df_link = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .option("mode", "DROPMALFORMED") \
    .load("hdfs://localhost:9000/user/bigdata2022/datasets/wiki/links.tsv")
df_link.show(5)

+--------------------+--------------+
|              source|   destination|
+--------------------+--------------+
|%C3%81ed%C3%A1n_m...|          Bede|
|%C3%81ed%C3%A1n_m...|       Columba|
|%C3%81ed%C3%A1n_m...|D%C3%A1l_Riata|
|%C3%81ed%C3%A1n_m...| Great_Britain|
|%C3%81ed%C3%A1n_m...|       Ireland|
+--------------------+--------------+
only showing top 5 rows



In [7]:
# Unqoute source and destination article
from urllib.parse import unquote
rdd_link = df_link.rdd
rdd_link  = rdd_link.map(lambda x: (unquote(x.source),unquote(x.destination)))
rdd_link.collect()

[('Áedán_mac_Gabráin', 'Bede'),
 ('Áedán_mac_Gabráin', 'Columba'),
 ('Áedán_mac_Gabráin', 'Dál_Riata'),
 ('Áedán_mac_Gabráin', 'Great_Britain'),
 ('Áedán_mac_Gabráin', 'Ireland'),
 ('Áedán_mac_Gabráin', 'Isle_of_Man'),
 ('Áedán_mac_Gabráin', 'Monarchy'),
 ('Áedán_mac_Gabráin', 'Orkney'),
 ('Áedán_mac_Gabráin', 'Picts'),
 ('Áedán_mac_Gabráin', 'Scotland'),
 ('Áedán_mac_Gabráin', 'Wales'),
 ('Åland', '20th_century'),
 ('Åland', 'Baltic_Sea'),
 ('Åland', 'Crimean_War'),
 ('Åland', 'Currency'),
 ('Åland', 'Euro'),
 ('Åland', 'European_Union'),
 ('Åland', 'Finland'),
 ('Åland', 'League_of_Nations'),
 ('Åland', 'List_of_countries_by_system_of_government'),
 ('Åland', 'Nationality'),
 ('Åland', 'Parliamentary_system'),
 ('Åland', 'Police'),
 ('Åland', 'Russia'),
 ('Åland', 'Stockholm'),
 ('Åland', 'Sweden'),
 ('Åland', 'Time_zone'),
 ('Åland', 'Tourism'),
 ('Åland', 'United_Kingdom'),
 ('Åland', 'World_War_II'),
 ('Édouard_Manet', 'Absinthe'),
 ('Édouard_Manet', 'Beer'),
 ('Édouard_Manet', 'C

In [9]:
# Read dataframe path_finished.tsv
df_path = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .option("mode", "DROPMALFORMED") \
    .load("hdfs://localhost:9000/user/bigdata2022/datasets/wiki/paths_finished.tsv")
df_path.show(5)

+----------------+----------+-------------+--------------------+------+
| hashedIpAddress| timestamp|durationInSec|                path|rating|
+----------------+----------+-------------+--------------------+------+
|6a3701d319fc3754|1297740409|          166|14th_century;15th...|  NULL|
|3824310e536af032|1344753412|           88|14th_century;Euro...|     3|
|415612e93584d30e|1349298640|          138|14th_century;Nige...|  NULL|
|64dd5cd342e3780c|1265613925|           37|14th_century;Rena...|  NULL|
|015245d773376aab|1366730828|          175|14th_century;Ital...|     3|
+----------------+----------+-------------+--------------------+------+
only showing top 5 rows



In [10]:
# Constructed a new rdd with the source and destination of the path, 
# the duration in seconds of the path, the rating of the path, 
# a variable (worth 0 if the rating is unknown or worth 1 if it is known) 
# and the length of the path.
def formatRow(x):
    values = unquote(x.path).split(";")
    length = len(values)
    if(x.rating == "NULL"):
        rating = 0
        ratingNum = 0
    else:
        rating = int(x.rating)
        ratingNum = 1
    return (values[0]+"::"+values[-1],int(x.durationInSec),rating,ratingNum,length)

rdd_path = df_path.rdd
rdd_path  = rdd_path.map(lambda x: formatRow(x))
rdd_path.collect()

[('14th_century::African_slave_trade', 166, 0, 0, 9),
 ('14th_century::African_slave_trade', 88, 3, 1, 5),
 ('14th_century::African_slave_trade', 138, 0, 0, 8),
 ('14th_century::Greece', 37, 0, 0, 4),
 ('14th_century::John_F._Kennedy', 175, 3, 1, 7),
 ('14th_century::John_F._Kennedy', 110, 0, 0, 6),
 ('14th_century::Fire', 112, 2, 1, 4),
 ('14th_century::Rainbow', 139, 1, 1, 6),
 ('14th_century::Rainbow', 74, 3, 1, 4),
 ('14th_century::Rainbow', 167, 0, 0, 7),
 ('14th_century::Rainbow', 253, 3, 1, 13),
 ('14th_century::Rainbow', 218, 3, 1, 10),
 ('14th_century::Rainbow', 66, 0, 0, 5),
 ('14th_century::Rainbow', 391, 5, 1, 16),
 ('14th_century::Rainbow', 432, 0, 0, 9),
 ('14th_century::Rainbow', 43, 0, 0, 5),
 ('14th_century::Rainbow', 387, 2, 1, 15),
 ('14th_century::Rainbow', 179, 3, 1, 10),
 ('14th_century::Rainbow', 169, 0, 0, 8),
 ('14th_century::Rainbow', 246, 0, 0, 14),
 ('14th_century::Rainbow', 345, 3, 1, 8),
 ('14th_century::Rainbow', 265, 4, 1, 7),
 ('14th_century::Rainbow', 

In [11]:
# Summed up the values of the rating variable for each distinct path
rdd_rating  = rdd_path.map(lambda x: (x[0],x[2])) \
                .reduceByKey(lambda a, b: a+b)
rdd_rating.collect()

[('14th_century::African_slave_trade', 3),
 ('14th_century::Rainbow', 34),
 ('14th_century::Sodium', 3),
 ('14th_century::Henry_David_Thoreau', 5),
 ('2004_Atlantic_hurricane_season::Ice_age', 3),
 ('2005_Atlantic_hurricane_season::Spain', 1),
 ('2005_Atlantic_hurricane_season::Jazz', 13),
 ('2005_Atlantic_hurricane_season::Liechtenstein', 7),
 ('2005_Atlantic_hurricane_season::Sweden', 0),
 ('2005_Atlantic_hurricane_season::Burundi', 2),
 ('2005_Atlantic_hurricane_season::Hydrogen_peroxide', 0),
 ('2005_Atlantic_hurricane_season::International_Court_of_Justice', 0),
 ('2005_Atlantic_hurricane_season::William_McKinley', 2),
 ('Aberdeen::Vampire_bat', 8),
 ('Aberdeen::Bread', 0),
 ('Aberdeen::Sparrowhawk', 0),
 ('Acceleration::Dresden', 2),
 ('Acceleration::Elephant', 1),
 ('Acceleration::Trout', 15),
 ('Acceleration::Potassium_iodide', 7),
 ('Achilles::Gold', 3),
 ('Achilles::Great_white_shark', 3),
 ('Achilles::Bongo_(antelope)', 2),
 ('Achilles_tendon::Great_white_shark', 2),
 ('Afri

In [12]:
# Summed up the number of user rating for each separate path
rdd_ratingsNum  = rdd_path.map(lambda x: (x[0],x[3])) \
                    .reduceByKey(lambda a, b: a+b)
rdd_ratingsNum.collect()

[('14th_century::African_slave_trade', 1),
 ('14th_century::Rainbow', 13),
 ('14th_century::Sodium', 1),
 ('14th_century::Henry_David_Thoreau', 1),
 ('2004_Atlantic_hurricane_season::Ice_age', 2),
 ('2005_Atlantic_hurricane_season::Spain', 1),
 ('2005_Atlantic_hurricane_season::Jazz', 4),
 ('2005_Atlantic_hurricane_season::Liechtenstein', 2),
 ('2005_Atlantic_hurricane_season::Sweden', 0),
 ('2005_Atlantic_hurricane_season::Burundi', 1),
 ('2005_Atlantic_hurricane_season::Hydrogen_peroxide', 0),
 ('2005_Atlantic_hurricane_season::International_Court_of_Justice', 0),
 ('2005_Atlantic_hurricane_season::William_McKinley', 1),
 ('Aberdeen::Vampire_bat', 2),
 ('Aberdeen::Bread', 0),
 ('Aberdeen::Sparrowhawk', 0),
 ('Acceleration::Dresden', 1),
 ('Acceleration::Elephant', 1),
 ('Acceleration::Trout', 4),
 ('Acceleration::Potassium_iodide', 2),
 ('Achilles::Gold', 1),
 ('Achilles::Great_white_shark', 1),
 ('Achilles::Bongo_(antelope)', 1),
 ('Achilles_tendon::Great_white_shark', 1),
 ('Africa

In [13]:
# Average rating of each individual path
def divide(x,y):
    return round(x/y) if y else 0 
    
rdd_rating2 = rdd_rating.union(rdd_ratingsNum).reduceByKey(lambda x,y : divide(x,y))
rdd_rating2.collect()

[('14th_century::African_slave_trade', 3),
 ('14th_century::Rainbow', 3),
 ('14th_century::Sodium', 3),
 ('14th_century::Henry_David_Thoreau', 5),
 ('2004_Atlantic_hurricane_season::Ice_age', 2),
 ('2005_Atlantic_hurricane_season::Spain', 1),
 ('2005_Atlantic_hurricane_season::Jazz', 3),
 ('2005_Atlantic_hurricane_season::Liechtenstein', 4),
 ('2005_Atlantic_hurricane_season::Sweden', 0),
 ('2005_Atlantic_hurricane_season::Burundi', 2),
 ('2005_Atlantic_hurricane_season::Hydrogen_peroxide', 0),
 ('2005_Atlantic_hurricane_season::International_Court_of_Justice', 0),
 ('2005_Atlantic_hurricane_season::William_McKinley', 2),
 ('Aberdeen::Vampire_bat', 4),
 ('Aberdeen::Bread', 0),
 ('Aberdeen::Sparrowhawk', 0),
 ('Acceleration::Dresden', 2),
 ('Acceleration::Elephant', 1),
 ('Acceleration::Trout', 4),
 ('Acceleration::Potassium_iodide', 4),
 ('Achilles::Gold', 3),
 ('Achilles::Great_white_shark', 3),
 ('Achilles::Bongo_(antelope)', 2),
 ('Achilles_tendon::Great_white_shark', 2),
 ('Africa:

In [14]:
# Average duration in seconds of a single path
import math

rdd_duration = rdd_path \
    .map(lambda x: (x[0], x[1])) \
    .mapValues(lambda v: (v, 1)) \
    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
    .mapValues(lambda v: math.ceil(v[0]/v[1]))
rdd_duration.collect()

[('14th_century::African_slave_trade', 131),
 ('14th_century::Rainbow', 182),
 ('14th_century::Sodium', 150),
 ('14th_century::Henry_David_Thoreau', 507),
 ('2004_Atlantic_hurricane_season::Ice_age', 109),
 ('2005_Atlantic_hurricane_season::Spain', 22),
 ('2005_Atlantic_hurricane_season::Jazz', 312),
 ('2005_Atlantic_hurricane_season::Liechtenstein', 309),
 ('2005_Atlantic_hurricane_season::Sweden', 53),
 ('2005_Atlantic_hurricane_season::Burundi', 148),
 ('2005_Atlantic_hurricane_season::Hydrogen_peroxide', 223),
 ('2005_Atlantic_hurricane_season::International_Court_of_Justice', 132),
 ('2005_Atlantic_hurricane_season::William_McKinley', 104),
 ('Aberdeen::Vampire_bat', 331),
 ('Aberdeen::Bread', 77),
 ('Aberdeen::Sparrowhawk', 206),
 ('Acceleration::Dresden', 59),
 ('Acceleration::Elephant', 40),
 ('Acceleration::Trout', 155),
 ('Acceleration::Potassium_iodide', 122),
 ('Achilles::Gold', 137),
 ('Achilles::Great_white_shark', 213),
 ('Achilles::Bongo_(antelope)', 159),
 ('Achilles_t

In [15]:
# Average length of each individual path
rdd_length = rdd_path \
    .map(lambda x: (x[0], x[4])) \
    .mapValues(lambda v: (v, 1)) \
    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
    .mapValues(lambda v: math.ceil(v[0]/v[1]))
rdd_length.collect()

[('14th_century::African_slave_trade', 8),
 ('14th_century::Rainbow', 8),
 ('14th_century::Sodium', 6),
 ('14th_century::Henry_David_Thoreau', 27),
 ('2004_Atlantic_hurricane_season::Ice_age', 7),
 ('2005_Atlantic_hurricane_season::Spain', 4),
 ('2005_Atlantic_hurricane_season::Jazz', 12),
 ('2005_Atlantic_hurricane_season::Liechtenstein', 5),
 ('2005_Atlantic_hurricane_season::Sweden', 5),
 ('2005_Atlantic_hurricane_season::Burundi', 4),
 ('2005_Atlantic_hurricane_season::Hydrogen_peroxide', 5),
 ('2005_Atlantic_hurricane_season::International_Court_of_Justice', 8),
 ('2005_Atlantic_hurricane_season::William_McKinley', 5),
 ('Aberdeen::Vampire_bat', 8),
 ('Aberdeen::Bread', 6),
 ('Aberdeen::Sparrowhawk', 8),
 ('Acceleration::Dresden', 4),
 ('Acceleration::Elephant', 4),
 ('Acceleration::Trout', 10),
 ('Acceleration::Potassium_iodide', 10),
 ('Achilles::Gold', 6),
 ('Achilles::Great_white_shark', 14),
 ('Achilles::Bongo_(antelope)', 6),
 ('Achilles_tendon::Great_white_shark', 10),
 ('A

In [16]:
# RDD containing the path, average rating and average duration in seconds  
rdd_final1 = rdd_rating2.join(rdd_duration)
rdd_final1.collect()

[('14th_century::African_slave_trade', (3, 131)),
 ('14th_century::Rainbow', (3, 182)),
 ('14th_century::Sodium', (3, 150)),
 ('14th_century::Henry_David_Thoreau', (5, 507)),
 ('2004_Atlantic_hurricane_season::Ice_age', (2, 109)),
 ('2005_Atlantic_hurricane_season::Spain', (1, 22)),
 ('2005_Atlantic_hurricane_season::Jazz', (3, 312)),
 ('2005_Atlantic_hurricane_season::Liechtenstein', (4, 309)),
 ('2005_Atlantic_hurricane_season::Sweden', (0, 53)),
 ('2005_Atlantic_hurricane_season::Burundi', (2, 148)),
 ('2005_Atlantic_hurricane_season::Hydrogen_peroxide', (0, 223)),
 ('2005_Atlantic_hurricane_season::International_Court_of_Justice', (0, 132)),
 ('2005_Atlantic_hurricane_season::William_McKinley', (2, 104)),
 ('Aberdeen::Vampire_bat', (4, 331)),
 ('Aberdeen::Bread', (0, 77)),
 ('Aberdeen::Sparrowhawk', (0, 206)),
 ('Acceleration::Dresden', (2, 59)),
 ('Acceleration::Elephant', (1, 40)),
 ('Acceleration::Trout', (4, 155)),
 ('Acceleration::Potassium_iodide', (4, 122)),
 ('Achilles::Gol

In [17]:
# RDD containing the path, average duration in seconds, average length and average rating
rdd_final2 = rdd_final1.join(rdd_length).map(lambda x: (x[0],x[1][0][1],x[1][1],x[1][0][0]))
rdd_final2.collect()

[('14th_century::African_slave_trade', 131, 8, 3),
 ('14th_century::Rainbow', 182, 8, 3),
 ('14th_century::Sodium', 150, 6, 3),
 ('14th_century::Henry_David_Thoreau', 507, 27, 5),
 ('2004_Atlantic_hurricane_season::Ice_age', 109, 7, 2),
 ('2005_Atlantic_hurricane_season::Spain', 22, 4, 1),
 ('2005_Atlantic_hurricane_season::Jazz', 312, 12, 3),
 ('2005_Atlantic_hurricane_season::Liechtenstein', 309, 5, 4),
 ('2005_Atlantic_hurricane_season::Sweden', 53, 5, 0),
 ('2005_Atlantic_hurricane_season::Burundi', 148, 4, 2),
 ('2005_Atlantic_hurricane_season::Hydrogen_peroxide', 223, 5, 0),
 ('2005_Atlantic_hurricane_season::International_Court_of_Justice', 132, 8, 0),
 ('2005_Atlantic_hurricane_season::William_McKinley', 104, 5, 2),
 ('Aberdeen::Vampire_bat', 331, 8, 4),
 ('Aberdeen::Bread', 77, 6, 0),
 ('Aberdeen::Sparrowhawk', 206, 8, 0),
 ('Acceleration::Dresden', 59, 4, 2),
 ('Acceleration::Elephant', 40, 4, 1),
 ('Acceleration::Trout', 155, 10, 4),
 ('Acceleration::Potassium_iodide', 122, 

In [18]:
# RDD containing the source path, the destination path, 
# average duration in seconds, average length and average rating
def splitRow(x):
    source, target = x[0].split("::")
    return (source, target, x[1], x[2], x[3])

rdd_final3  = rdd_final2.map(lambda x: splitRow(x))
rdd_final3.collect()

[('14th_century', 'African_slave_trade', 131, 8, 3),
 ('14th_century', 'Rainbow', 182, 8, 3),
 ('14th_century', 'Sodium', 150, 6, 3),
 ('14th_century', 'Henry_David_Thoreau', 507, 27, 5),
 ('2004_Atlantic_hurricane_season', 'Ice_age', 109, 7, 2),
 ('2005_Atlantic_hurricane_season', 'Spain', 22, 4, 1),
 ('2005_Atlantic_hurricane_season', 'Jazz', 312, 12, 3),
 ('2005_Atlantic_hurricane_season', 'Liechtenstein', 309, 5, 4),
 ('2005_Atlantic_hurricane_season', 'Sweden', 53, 5, 0),
 ('2005_Atlantic_hurricane_season', 'Burundi', 148, 4, 2),
 ('2005_Atlantic_hurricane_season', 'Hydrogen_peroxide', 223, 5, 0),
 ('2005_Atlantic_hurricane_season',
  'International_Court_of_Justice',
  132,
  8,
  0),
 ('2005_Atlantic_hurricane_season', 'William_McKinley', 104, 5, 2),
 ('Aberdeen', 'Vampire_bat', 331, 8, 4),
 ('Aberdeen', 'Bread', 77, 6, 0),
 ('Aberdeen', 'Sparrowhawk', 206, 8, 0),
 ('Acceleration', 'Dresden', 59, 4, 2),
 ('Acceleration', 'Elephant', 40, 4, 1),
 ('Acceleration', 'Trout', 155, 10,

In [19]:
# Convert RDD to DF

df_path = rdd_final3.toDF(["source","target","duration","pathLength","rating"])
#df_path.printSchema()
#df_path.show(truncate=False)

df_cat = rdd_cat.toDF(["source","category"])
#df_cat.printSchema()
#df_cat.show(truncate=False)

df_link = rdd_link.toDF(["source","target"])
#df_link.printSchema()
#df_link.show(truncate=False)

In [20]:
#df_path.write.option("header",True) \
# .csv("hdfs://localhost:9000/user/bigdata2022/output/wiki/paths.csv")

#df_cat.write.option("header",True) \
# .csv("hdfs://localhost:9000/user/bigdata2022/output/wiki/categories.csv")

#df_link.write.option("header",True) \
# .csv("hdfs://localhost:9000/user/bigdata2022/output/wiki/links.csv")

In [21]:
# Inserted article nodes, with their respective categories, within the neo4j database
from pyspark.sql.functions import col
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", "bigdata"))

source = df_cat.select('source').rdd.flatMap(lambda x: x).collect()
category = df_cat.select('category').rdd.flatMap(lambda x: x).collect()

length = len(source)

for i in range(length):
    print(str(i) +" on "+str(length))
    graph.run('CREATE (n:Article {name: "'+source[i]+'", category: "'+str(category[i])+'"})')

0 on 4598
1 on 4598
2 on 4598
3 on 4598
4 on 4598
5 on 4598
6 on 4598
7 on 4598
8 on 4598
9 on 4598
10 on 4598
11 on 4598
12 on 4598
13 on 4598
14 on 4598
15 on 4598
16 on 4598
17 on 4598
18 on 4598
19 on 4598
20 on 4598
21 on 4598
22 on 4598
23 on 4598
24 on 4598
25 on 4598
26 on 4598
27 on 4598
28 on 4598
29 on 4598
30 on 4598
31 on 4598
32 on 4598
33 on 4598
34 on 4598
35 on 4598
36 on 4598
37 on 4598
38 on 4598
39 on 4598
40 on 4598
41 on 4598
42 on 4598
43 on 4598
44 on 4598
45 on 4598
46 on 4598
47 on 4598
48 on 4598
49 on 4598
50 on 4598
51 on 4598
52 on 4598
53 on 4598
54 on 4598
55 on 4598
56 on 4598
57 on 4598
58 on 4598
59 on 4598
60 on 4598
61 on 4598
62 on 4598
63 on 4598
64 on 4598
65 on 4598
66 on 4598
67 on 4598
68 on 4598
69 on 4598
70 on 4598
71 on 4598
72 on 4598
73 on 4598
74 on 4598
75 on 4598
76 on 4598
77 on 4598
78 on 4598
79 on 4598
80 on 4598
81 on 4598
82 on 4598
83 on 4598
84 on 4598
85 on 4598
86 on 4598
87 on 4598
88 on 4598
89 on 4598
90 on 4598
91 on 459

701 on 4598
702 on 4598
703 on 4598
704 on 4598
705 on 4598
706 on 4598
707 on 4598
708 on 4598
709 on 4598
710 on 4598
711 on 4598
712 on 4598
713 on 4598
714 on 4598
715 on 4598
716 on 4598
717 on 4598
718 on 4598
719 on 4598
720 on 4598
721 on 4598
722 on 4598
723 on 4598
724 on 4598
725 on 4598
726 on 4598
727 on 4598
728 on 4598
729 on 4598
730 on 4598
731 on 4598
732 on 4598
733 on 4598
734 on 4598
735 on 4598
736 on 4598
737 on 4598
738 on 4598
739 on 4598
740 on 4598
741 on 4598
742 on 4598
743 on 4598
744 on 4598
745 on 4598
746 on 4598
747 on 4598
748 on 4598
749 on 4598
750 on 4598
751 on 4598
752 on 4598
753 on 4598
754 on 4598
755 on 4598
756 on 4598
757 on 4598
758 on 4598
759 on 4598
760 on 4598
761 on 4598
762 on 4598
763 on 4598
764 on 4598
765 on 4598
766 on 4598
767 on 4598
768 on 4598
769 on 4598
770 on 4598
771 on 4598
772 on 4598
773 on 4598
774 on 4598
775 on 4598
776 on 4598
777 on 4598
778 on 4598
779 on 4598
780 on 4598
781 on 4598
782 on 4598
783 on 4598
784 

1388 on 4598
1389 on 4598
1390 on 4598
1391 on 4598
1392 on 4598
1393 on 4598
1394 on 4598
1395 on 4598
1396 on 4598
1397 on 4598
1398 on 4598
1399 on 4598
1400 on 4598
1401 on 4598
1402 on 4598
1403 on 4598
1404 on 4598
1405 on 4598
1406 on 4598
1407 on 4598
1408 on 4598
1409 on 4598
1410 on 4598
1411 on 4598
1412 on 4598
1413 on 4598
1414 on 4598
1415 on 4598
1416 on 4598
1417 on 4598
1418 on 4598
1419 on 4598
1420 on 4598
1421 on 4598
1422 on 4598
1423 on 4598
1424 on 4598
1425 on 4598
1426 on 4598
1427 on 4598
1428 on 4598
1429 on 4598
1430 on 4598
1431 on 4598
1432 on 4598
1433 on 4598
1434 on 4598
1435 on 4598
1436 on 4598
1437 on 4598
1438 on 4598
1439 on 4598
1440 on 4598
1441 on 4598
1442 on 4598
1443 on 4598
1444 on 4598
1445 on 4598
1446 on 4598
1447 on 4598
1448 on 4598
1449 on 4598
1450 on 4598
1451 on 4598
1452 on 4598
1453 on 4598
1454 on 4598
1455 on 4598
1456 on 4598
1457 on 4598
1458 on 4598
1459 on 4598
1460 on 4598
1461 on 4598
1462 on 4598
1463 on 4598
1464 on 4598

2031 on 4598
2032 on 4598
2033 on 4598
2034 on 4598
2035 on 4598
2036 on 4598
2037 on 4598
2038 on 4598
2039 on 4598
2040 on 4598
2041 on 4598
2042 on 4598
2043 on 4598
2044 on 4598
2045 on 4598
2046 on 4598
2047 on 4598
2048 on 4598
2049 on 4598
2050 on 4598
2051 on 4598
2052 on 4598
2053 on 4598
2054 on 4598
2055 on 4598
2056 on 4598
2057 on 4598
2058 on 4598
2059 on 4598
2060 on 4598
2061 on 4598
2062 on 4598
2063 on 4598
2064 on 4598
2065 on 4598
2066 on 4598
2067 on 4598
2068 on 4598
2069 on 4598
2070 on 4598
2071 on 4598
2072 on 4598
2073 on 4598
2074 on 4598
2075 on 4598
2076 on 4598
2077 on 4598
2078 on 4598
2079 on 4598
2080 on 4598
2081 on 4598
2082 on 4598
2083 on 4598
2084 on 4598
2085 on 4598
2086 on 4598
2087 on 4598
2088 on 4598
2089 on 4598
2090 on 4598
2091 on 4598
2092 on 4598
2093 on 4598
2094 on 4598
2095 on 4598
2096 on 4598
2097 on 4598
2098 on 4598
2099 on 4598
2100 on 4598
2101 on 4598
2102 on 4598
2103 on 4598
2104 on 4598
2105 on 4598
2106 on 4598
2107 on 4598

2684 on 4598
2685 on 4598
2686 on 4598
2687 on 4598
2688 on 4598
2689 on 4598
2690 on 4598
2691 on 4598
2692 on 4598
2693 on 4598
2694 on 4598
2695 on 4598
2696 on 4598
2697 on 4598
2698 on 4598
2699 on 4598
2700 on 4598
2701 on 4598
2702 on 4598
2703 on 4598
2704 on 4598
2705 on 4598
2706 on 4598
2707 on 4598
2708 on 4598
2709 on 4598
2710 on 4598
2711 on 4598
2712 on 4598
2713 on 4598
2714 on 4598
2715 on 4598
2716 on 4598
2717 on 4598
2718 on 4598
2719 on 4598
2720 on 4598
2721 on 4598
2722 on 4598
2723 on 4598
2724 on 4598
2725 on 4598
2726 on 4598
2727 on 4598
2728 on 4598
2729 on 4598
2730 on 4598
2731 on 4598
2732 on 4598
2733 on 4598
2734 on 4598
2735 on 4598
2736 on 4598
2737 on 4598
2738 on 4598
2739 on 4598
2740 on 4598
2741 on 4598
2742 on 4598
2743 on 4598
2744 on 4598
2745 on 4598
2746 on 4598
2747 on 4598
2748 on 4598
2749 on 4598
2750 on 4598
2751 on 4598
2752 on 4598
2753 on 4598
2754 on 4598
2755 on 4598
2756 on 4598
2757 on 4598
2758 on 4598
2759 on 4598
2760 on 4598

3333 on 4598
3334 on 4598
3335 on 4598
3336 on 4598
3337 on 4598
3338 on 4598
3339 on 4598
3340 on 4598
3341 on 4598
3342 on 4598
3343 on 4598
3344 on 4598
3345 on 4598
3346 on 4598
3347 on 4598
3348 on 4598
3349 on 4598
3350 on 4598
3351 on 4598
3352 on 4598
3353 on 4598
3354 on 4598
3355 on 4598
3356 on 4598
3357 on 4598
3358 on 4598
3359 on 4598
3360 on 4598
3361 on 4598
3362 on 4598
3363 on 4598
3364 on 4598
3365 on 4598
3366 on 4598
3367 on 4598
3368 on 4598
3369 on 4598
3370 on 4598
3371 on 4598
3372 on 4598
3373 on 4598
3374 on 4598
3375 on 4598
3376 on 4598
3377 on 4598
3378 on 4598
3379 on 4598
3380 on 4598
3381 on 4598
3382 on 4598
3383 on 4598
3384 on 4598
3385 on 4598
3386 on 4598
3387 on 4598
3388 on 4598
3389 on 4598
3390 on 4598
3391 on 4598
3392 on 4598
3393 on 4598
3394 on 4598
3395 on 4598
3396 on 4598
3397 on 4598
3398 on 4598
3399 on 4598
3400 on 4598
3401 on 4598
3402 on 4598
3403 on 4598
3404 on 4598
3405 on 4598
3406 on 4598
3407 on 4598
3408 on 4598
3409 on 4598

3995 on 4598
3996 on 4598
3997 on 4598
3998 on 4598
3999 on 4598
4000 on 4598
4001 on 4598
4002 on 4598
4003 on 4598
4004 on 4598
4005 on 4598
4006 on 4598
4007 on 4598
4008 on 4598
4009 on 4598
4010 on 4598
4011 on 4598
4012 on 4598
4013 on 4598
4014 on 4598
4015 on 4598
4016 on 4598
4017 on 4598
4018 on 4598
4019 on 4598
4020 on 4598
4021 on 4598
4022 on 4598
4023 on 4598
4024 on 4598
4025 on 4598
4026 on 4598
4027 on 4598
4028 on 4598
4029 on 4598
4030 on 4598
4031 on 4598
4032 on 4598
4033 on 4598
4034 on 4598
4035 on 4598
4036 on 4598
4037 on 4598
4038 on 4598
4039 on 4598
4040 on 4598
4041 on 4598
4042 on 4598
4043 on 4598
4044 on 4598
4045 on 4598
4046 on 4598
4047 on 4598
4048 on 4598
4049 on 4598
4050 on 4598
4051 on 4598
4052 on 4598
4053 on 4598
4054 on 4598
4055 on 4598
4056 on 4598
4057 on 4598
4058 on 4598
4059 on 4598
4060 on 4598
4061 on 4598
4062 on 4598
4063 on 4598
4064 on 4598
4065 on 4598
4066 on 4598
4067 on 4598
4068 on 4598
4069 on 4598
4070 on 4598
4071 on 4598

In [22]:
# Inserted direct edges between two different articles (hyperlink) within the Neo4j database
source = df_link.select('source').rdd.flatMap(lambda x: x).collect()
target = df_link.select('target').rdd.flatMap(lambda x: x).collect()

length = len(source)

for i in range(length):
    print(str(i) +" on "+str(length))
    graph.run('MATCH (a:Article), (b:Article) WHERE a.name = "'+source[i]+'" AND b.name = "'+target[i]+'"'+
              'CREATE (a)-[r:HYPERLINK]->(b) RETURN type(r)')

In [23]:
# Inserted direct edges between two different articles of a path (challenge), 
# with the respective average rating, average duration in seconds and 
# average length in seconds, within the Neo4j database
source = df_path.select('source').rdd.flatMap(lambda x: x).collect()
target = df_path.select('target').rdd.flatMap(lambda x: x).collect()
duration = df_path.select('duration').rdd.flatMap(lambda x: x).collect()
pathLength = df_path.select('pathLength').rdd.flatMap(lambda x: x).collect()
rating = df_path.select('rating').rdd.flatMap(lambda x: x).collect()

length = len(source)

for i in range(length):
    print(str(i) +" on "+str(length))
    graph.run('MATCH (a:Article), (b:Article) WHERE a.name = "'+source[i]+'" AND b.name = "'+target[i]+'"'+
              'CREATE (a)-[r:CHALLENGE]->(b) '+
              'SET r.duration = "'+str(duration[i])+'", '+
              'r.pathLength = "'+str(pathLength[i])+'", '+
              'r.rating = "'+str(rating[i])+'"')

0 on 28718
1 on 28718
2 on 28718
3 on 28718
4 on 28718
5 on 28718
6 on 28718
7 on 28718
8 on 28718
9 on 28718
10 on 28718
11 on 28718
12 on 28718
13 on 28718
14 on 28718
15 on 28718
16 on 28718
17 on 28718
18 on 28718
19 on 28718
20 on 28718
21 on 28718
22 on 28718
23 on 28718
24 on 28718
25 on 28718
26 on 28718
27 on 28718
28 on 28718
29 on 28718
30 on 28718
31 on 28718
32 on 28718
33 on 28718
34 on 28718
35 on 28718
36 on 28718
37 on 28718
38 on 28718
39 on 28718
40 on 28718
41 on 28718
42 on 28718
43 on 28718
44 on 28718
45 on 28718
46 on 28718
47 on 28718
48 on 28718
49 on 28718
50 on 28718
51 on 28718
52 on 28718
53 on 28718
54 on 28718
55 on 28718
56 on 28718
57 on 28718
58 on 28718
59 on 28718
60 on 28718
61 on 28718
62 on 28718
63 on 28718
64 on 28718
65 on 28718
66 on 28718
67 on 28718
68 on 28718
69 on 28718
70 on 28718
71 on 28718
72 on 28718
73 on 28718
74 on 28718
75 on 28718
76 on 28718
77 on 28718
78 on 28718
79 on 28718
80 on 28718
81 on 28718
82 on 28718
83 on 28718
84

649 on 28718
650 on 28718
651 on 28718
652 on 28718
653 on 28718
654 on 28718
655 on 28718
656 on 28718
657 on 28718
658 on 28718
659 on 28718
660 on 28718
661 on 28718
662 on 28718
663 on 28718
664 on 28718
665 on 28718
666 on 28718
667 on 28718
668 on 28718
669 on 28718
670 on 28718
671 on 28718
672 on 28718
673 on 28718
674 on 28718
675 on 28718
676 on 28718
677 on 28718
678 on 28718
679 on 28718
680 on 28718
681 on 28718
682 on 28718
683 on 28718
684 on 28718
685 on 28718
686 on 28718
687 on 28718
688 on 28718
689 on 28718
690 on 28718
691 on 28718
692 on 28718
693 on 28718
694 on 28718
695 on 28718
696 on 28718
697 on 28718
698 on 28718
699 on 28718
700 on 28718
701 on 28718
702 on 28718
703 on 28718
704 on 28718
705 on 28718
706 on 28718
707 on 28718
708 on 28718
709 on 28718
710 on 28718
711 on 28718
712 on 28718
713 on 28718
714 on 28718
715 on 28718
716 on 28718
717 on 28718
718 on 28718
719 on 28718
720 on 28718
721 on 28718
722 on 28718
723 on 28718
724 on 28718
725 on 28718

1262 on 28718
1263 on 28718
1264 on 28718
1265 on 28718
1266 on 28718
1267 on 28718
1268 on 28718
1269 on 28718
1270 on 28718
1271 on 28718
1272 on 28718
1273 on 28718
1274 on 28718
1275 on 28718
1276 on 28718
1277 on 28718
1278 on 28718
1279 on 28718
1280 on 28718
1281 on 28718
1282 on 28718
1283 on 28718
1284 on 28718
1285 on 28718
1286 on 28718
1287 on 28718
1288 on 28718
1289 on 28718
1290 on 28718
1291 on 28718
1292 on 28718
1293 on 28718
1294 on 28718
1295 on 28718
1296 on 28718
1297 on 28718
1298 on 28718
1299 on 28718
1300 on 28718
1301 on 28718
1302 on 28718
1303 on 28718
1304 on 28718
1305 on 28718
1306 on 28718
1307 on 28718
1308 on 28718
1309 on 28718
1310 on 28718
1311 on 28718
1312 on 28718
1313 on 28718
1314 on 28718
1315 on 28718
1316 on 28718
1317 on 28718
1318 on 28718
1319 on 28718
1320 on 28718
1321 on 28718
1322 on 28718
1323 on 28718
1324 on 28718
1325 on 28718
1326 on 28718
1327 on 28718
1328 on 28718
1329 on 28718
1330 on 28718
1331 on 28718
1332 on 28718
1333 o

1857 on 28718
1858 on 28718
1859 on 28718
1860 on 28718
1861 on 28718
1862 on 28718
1863 on 28718
1864 on 28718
1865 on 28718
1866 on 28718
1867 on 28718
1868 on 28718
1869 on 28718
1870 on 28718
1871 on 28718
1872 on 28718
1873 on 28718
1874 on 28718
1875 on 28718
1876 on 28718
1877 on 28718
1878 on 28718
1879 on 28718
1880 on 28718
1881 on 28718
1882 on 28718
1883 on 28718
1884 on 28718
1885 on 28718
1886 on 28718
1887 on 28718
1888 on 28718
1889 on 28718
1890 on 28718
1891 on 28718
1892 on 28718
1893 on 28718
1894 on 28718
1895 on 28718
1896 on 28718
1897 on 28718
1898 on 28718
1899 on 28718
1900 on 28718
1901 on 28718
1902 on 28718
1903 on 28718
1904 on 28718
1905 on 28718
1906 on 28718
1907 on 28718
1908 on 28718
1909 on 28718
1910 on 28718
1911 on 28718
1912 on 28718
1913 on 28718
1914 on 28718
1915 on 28718
1916 on 28718
1917 on 28718
1918 on 28718
1919 on 28718
1920 on 28718
1921 on 28718
1922 on 28718
1923 on 28718
1924 on 28718
1925 on 28718
1926 on 28718
1927 on 28718
1928 o

2456 on 28718
2457 on 28718
2458 on 28718
2459 on 28718
2460 on 28718
2461 on 28718
2462 on 28718
2463 on 28718
2464 on 28718
2465 on 28718
2466 on 28718
2467 on 28718
2468 on 28718
2469 on 28718
2470 on 28718
2471 on 28718
2472 on 28718
2473 on 28718
2474 on 28718
2475 on 28718
2476 on 28718
2477 on 28718
2478 on 28718
2479 on 28718
2480 on 28718
2481 on 28718
2482 on 28718
2483 on 28718
2484 on 28718
2485 on 28718
2486 on 28718
2487 on 28718
2488 on 28718
2489 on 28718
2490 on 28718
2491 on 28718
2492 on 28718
2493 on 28718
2494 on 28718
2495 on 28718
2496 on 28718
2497 on 28718
2498 on 28718
2499 on 28718
2500 on 28718
2501 on 28718
2502 on 28718
2503 on 28718
2504 on 28718
2505 on 28718
2506 on 28718
2507 on 28718
2508 on 28718
2509 on 28718
2510 on 28718
2511 on 28718
2512 on 28718
2513 on 28718
2514 on 28718
2515 on 28718
2516 on 28718
2517 on 28718
2518 on 28718
2519 on 28718
2520 on 28718
2521 on 28718
2522 on 28718
2523 on 28718
2524 on 28718
2525 on 28718
2526 on 28718
2527 o

...

26204 on 28718
26205 on 28718
26206 on 28718
26207 on 28718
26208 on 28718
26209 on 28718
26210 on 28718
26211 on 28718
26212 on 28718
26213 on 28718
26214 on 28718
26215 on 28718
26216 on 28718
26217 on 28718
26218 on 28718
26219 on 28718
26220 on 28718
26221 on 28718
26222 on 28718
26223 on 28718
26224 on 28718
26225 on 28718
26226 on 28718
26227 on 28718
26228 on 28718
26229 on 28718
26230 on 28718
26231 on 28718
26232 on 28718
26233 on 28718
26234 on 28718
26235 on 28718
26236 on 28718
26237 on 28718
26238 on 28718
26239 on 28718
26240 on 28718
26241 on 28718
26242 on 28718
26243 on 28718
26244 on 28718
26245 on 28718
26246 on 28718
26247 on 28718
26248 on 28718
26249 on 28718
26250 on 28718
26251 on 28718
26252 on 28718
26253 on 28718
26254 on 28718
26255 on 28718
26256 on 28718
26257 on 28718
26258 on 28718
26259 on 28718
26260 on 28718
26261 on 28718
26262 on 28718
26263 on 28718
26264 on 28718
26265 on 28718
26266 on 28718
26267 on 28718
26268 on 28718
26269 on 28718
26270 on 2

26759 on 28718
26760 on 28718
26761 on 28718
26762 on 28718
26763 on 28718
26764 on 28718
26765 on 28718
26766 on 28718
26767 on 28718
26768 on 28718
26769 on 28718
26770 on 28718
26771 on 28718
26772 on 28718
26773 on 28718
26774 on 28718
26775 on 28718
26776 on 28718
26777 on 28718
26778 on 28718
26779 on 28718
26780 on 28718
26781 on 28718
26782 on 28718
26783 on 28718
26784 on 28718
26785 on 28718
26786 on 28718
26787 on 28718
26788 on 28718
26789 on 28718
26790 on 28718
26791 on 28718
26792 on 28718
26793 on 28718
26794 on 28718
26795 on 28718
26796 on 28718
26797 on 28718
26798 on 28718
26799 on 28718
26800 on 28718
26801 on 28718
26802 on 28718
26803 on 28718
26804 on 28718
26805 on 28718
26806 on 28718
26807 on 28718
26808 on 28718
26809 on 28718
26810 on 28718
26811 on 28718
26812 on 28718
26813 on 28718
26814 on 28718
26815 on 28718
26816 on 28718
26817 on 28718
26818 on 28718
26819 on 28718
26820 on 28718
26821 on 28718
26822 on 28718
26823 on 28718
26824 on 28718
26825 on 2

27308 on 28718
27309 on 28718
27310 on 28718
27311 on 28718
27312 on 28718
27313 on 28718
27314 on 28718
27315 on 28718
27316 on 28718
27317 on 28718
27318 on 28718
27319 on 28718
27320 on 28718
27321 on 28718
27322 on 28718
27323 on 28718
27324 on 28718
27325 on 28718
27326 on 28718
27327 on 28718
27328 on 28718
27329 on 28718
27330 on 28718
27331 on 28718
27332 on 28718
27333 on 28718
27334 on 28718
27335 on 28718
27336 on 28718
27337 on 28718
27338 on 28718
27339 on 28718
27340 on 28718
27341 on 28718
27342 on 28718
27343 on 28718
27344 on 28718
27345 on 28718
27346 on 28718
27347 on 28718
27348 on 28718
27349 on 28718
27350 on 28718
27351 on 28718
27352 on 28718
27353 on 28718
27354 on 28718
27355 on 28718
27356 on 28718
27357 on 28718
27358 on 28718
27359 on 28718
27360 on 28718
27361 on 28718
27362 on 28718
27363 on 28718
27364 on 28718
27365 on 28718
27366 on 28718
27367 on 28718
27368 on 28718
27369 on 28718
27370 on 28718
27371 on 28718
27372 on 28718
27373 on 28718
27374 on 2

27855 on 28718
27856 on 28718
27857 on 28718
27858 on 28718
27859 on 28718
27860 on 28718
27861 on 28718
27862 on 28718
27863 on 28718
27864 on 28718
27865 on 28718
27866 on 28718
27867 on 28718
27868 on 28718
27869 on 28718
27870 on 28718
27871 on 28718
27872 on 28718
27873 on 28718
27874 on 28718
27875 on 28718
27876 on 28718
27877 on 28718
27878 on 28718
27879 on 28718
27880 on 28718
27881 on 28718
27882 on 28718
27883 on 28718
27884 on 28718
27885 on 28718
27886 on 28718
27887 on 28718
27888 on 28718
27889 on 28718
27890 on 28718
27891 on 28718
27892 on 28718
27893 on 28718
27894 on 28718
27895 on 28718
27896 on 28718
27897 on 28718
27898 on 28718
27899 on 28718
27900 on 28718
27901 on 28718
27902 on 28718
27903 on 28718
27904 on 28718
27905 on 28718
27906 on 28718
27907 on 28718
27908 on 28718
27909 on 28718
27910 on 28718
27911 on 28718
27912 on 28718
27913 on 28718
27914 on 28718
27915 on 28718
27916 on 28718
27917 on 28718
27918 on 28718
27919 on 28718
27920 on 28718
27921 on 2

28405 on 28718
28406 on 28718
28407 on 28718
28408 on 28718
28409 on 28718
28410 on 28718
28411 on 28718
28412 on 28718
28413 on 28718
28414 on 28718
28415 on 28718
28416 on 28718
28417 on 28718
28418 on 28718
28419 on 28718
28420 on 28718
28421 on 28718
28422 on 28718
28423 on 28718
28424 on 28718
28425 on 28718
28426 on 28718
28427 on 28718
28428 on 28718
28429 on 28718
28430 on 28718
28431 on 28718
28432 on 28718
28433 on 28718
28434 on 28718
28435 on 28718
28436 on 28718
28437 on 28718
28438 on 28718
28439 on 28718
28440 on 28718
28441 on 28718
28442 on 28718
28443 on 28718
28444 on 28718
28445 on 28718
28446 on 28718
28447 on 28718
28448 on 28718
28449 on 28718
28450 on 28718
28451 on 28718
28452 on 28718
28453 on 28718
28454 on 28718
28455 on 28718
28456 on 28718
28457 on 28718
28458 on 28718
28459 on 28718
28460 on 28718
28461 on 28718
28462 on 28718
28463 on 28718
28464 on 28718
28465 on 28718
28466 on 28718
28467 on 28718
28468 on 28718
28469 on 28718
28470 on 28718
28471 on 2

In [24]:
# Stop context
spark.stop()